4.0 Feature Engineering and Data Preprocessing
We perform feature engineering and data preprocessing by:

Adding Technical Indicators to the data. The technical inicators are used as inputs in the training of our Reinforcement Learning Model
Adding Coveriance Matrices which is also used as input for training the Models

In [19]:
##Feature engineering

In [20]:
import pandas as pd
import numpy as np
import ta
from ta import add_all_ta_features
from ta.utils import dropna


In [21]:
# Load the whole data set
data = pd.read_csv('datasets/data.csv')

# Load the close prices dataset
prices_data = pd.read_csv('datasets/close_prices.csv')

In [22]:
%store -r filtered_stocks

In [23]:
list_of_stocks = filtered_stocks
print(list_of_stocks)

Index(['COST', 'PG', 'HD', 'GOOG', 'GOOGL', 'V', 'MSFT', 'WMT', 'AAPL', 'AMZN',
       'ORCL', 'MA', 'UNH', 'JPM', 'XOM', 'LLY', 'AVGO', 'TSLA', 'META',
       'NVDA'],
      dtype='object', name='stock_name')


In [24]:
filtered_stocks

Index(['COST', 'PG', 'HD', 'GOOG', 'GOOGL', 'V', 'MSFT', 'WMT', 'AAPL', 'AMZN',
       'ORCL', 'MA', 'UNH', 'JPM', 'XOM', 'LLY', 'AVGO', 'TSLA', 'META',
       'NVDA'],
      dtype='object', name='stock_name')

In [25]:
data

,date,tick,close,high,low,open,volume
0,2013-01-02 00:00:00+00:00,AAPL,16.687344,19.821428,19.343929,19.779285,560518000
1,2013-01-02 00:00:00+00:00,AMZN,12.865500,12.905000,12.663000,12.804000,65420000
2,2013-01-02 00:00:00+00:00,AVGO,2.395675,3.306000,3.224000,3.243000,36716000
3,2013-01-02 00:00:00+00:00,COST,80.408737,101.449997,100.209999,100.599998,3153800
4,2013-01-02 00:00:00+00:00,GOOG,17.969599,18.107130,17.846855,17.918339,102033017
...,...,...,...,...,...,...,...
55355,2023-12-29 00:00:00+00:00,TSLA,248.479996,255.190002,247.429993,255.100006,100615300
55356,2023-12-29 00:00:00+00:00,UNH,520.342346,528.239990,523.919983,525.979980,2080100
55357,2023-12-29 00:00:00+00:00,V,258.375061,261.459991,259.670013,260.570007,4074600
55358,2023-12-29 00:00:00+00:00,WMT,52.042637,52.693333,52.386665,52.509998,21948300


In [26]:
data = data[data['tick'].isin(list_of_stocks)]

In [27]:
data.tick.unique()

array(['AAPL', 'AMZN', 'AVGO', 'COST', 'GOOG', 'GOOGL', 'HD', 'JPM',
       'LLY', 'MA', 'META', 'MSFT', 'NVDA', 'ORCL', 'PG', 'TSLA', 'UNH',
       'V', 'WMT', 'XOM'], dtype=object)

In [28]:
# # Filtering top 1% volume outliers
# data = data[data['volume'] < data['volume'].quantile(0.99)].copy()  # Make sure we are working on a copy

# # Scaling down volume by a factor
# data.loc[:, 'volume'] = data['volume'] / 1000  # Use .loc to avoid SettingWithCopyWarning

In [29]:
## Adding Technical Indicators
# Volatility Average True Range (ATR)
#Volatility Bollinger Band Width (BBW)
#Volume On-balance Volume (OBV
#Volume Chaikin Money Flow (CMF)
#Trend Moving Average Convergence Divergence (MACD)
#Trend Average Directional Index (ADX)
#Trend Fast Simple Moving Average (SMA)
#Trend Fast Exponential Moving Average (EMA)
#Trend Commodity Channel Index (CCI)
#Momentum Relative Strength Index (RSI)

In [30]:
# List of Features to add (FinRL's required indicators)
feature_list = [
    'trend_macd',                  # MACD
    'volatility_bollinger_hband',  # Bollinger Bands Upper
    'volatility_bollinger_lband',  # Bollinger Bands Lower
    'momentum_rsi',                # RSI with 30-day window
    'trend_cci',                   # CCI with 30-day window
    'trend_dx',                    # DX with 30-day window
    'trend_sma_fast',              # Close 30 SMA
    'trend_sma_slow'               # Close 60 SMA
]

# Short names corresponding to the required indicators
short_names = [
    'macd',          # MACD
    'boll_ub',       # Bollinger Bands Upper
    'boll_lb',       # Bollinger Bands Lower
    'rsi_30',        # RSI 30
    'cci_30',        # CCI 30
    'dx_30',         # DX 30
    'close_30_sma',  # Close 30 SMA
    'close_60_sma'   # Close 60 SMA
]

In [43]:
import ta

def add_features(data, feature_list, short_names):
    """
    Function to add technical indicators for features.
    - Takes in a dataset with Open, High, Low, Close, and Volume.
    - Adds specified technical indicators and renames them.
    """
    
    # Original columns
    data_col_names = list(data.columns)
    
    # Desired columns after adding indicators
    filter_names = data_col_names + feature_list
    col_rename = data_col_names + short_names
    
    # 1. MACD
    macd = ta.trend.MACD(close=data['close'])
    data['trend_macd'] = macd.macd()
    
    # 2. Bollinger Bands Upper and Lower
    bollinger = ta.volatility.BollingerBands(close=data['close'], window=20, window_dev=2)
    data['volatility_bollinger_hband'] = bollinger.bollinger_hband()
    data['volatility_bollinger_lband'] = bollinger.bollinger_lband()
    
    # 3. RSI with 30-day window
    rsi = ta.momentum.RSIIndicator(close=data['close'], window=30)
    data['momentum_rsi'] = rsi.rsi()
    
    # 4. CCI with 30-day window
    cci = ta.trend.CCIIndicator(high=data['high'], low=data['low'], close=data['close'], window=30)
    data['trend_cci'] = cci.cci()
    
    # 5. ADX and DX with 30-day window
    adx = ta.trend.ADXIndicator(high=data['high'], low=data['low'], close=data['close'], window=30)
    data['trend_dx'] = adx.adx()  # DX_30
    # If you need ADX as well, uncomment the next line:
    # data['trend_adx'] = adx.adx()
    
    # 6. Close 30 SMA
    sma_30 = ta.trend.SMAIndicator(close=data['close'], window=30)
    data['trend_sma_fast'] = sma_30.sma_indicator()
    
    # 7. Close 60 SMA
    sma_60 = ta.trend.SMAIndicator(close=data['close'], window=60)
    data['trend_sma_slow'] = sma_60.sma_indicator()
    
    # Filter the DataFrame to include only desired columns
    data = data[filter_names]
    
    # Rename the indicator columns to match FinRL's expected names
    data.columns = col_rename
    
    # Drop any rows with NaN values resulting from indicator calculations
    data = data.dropna()
    
    return data


In [45]:
# Add Indicators to your dataset
data_with_features = data.copy()
data_with_features = add_features(data_with_features, feature_list, short_names)

# Verify the added indicators
print("Data with Features Columns:")
print(data_with_features.columns.tolist())


Data with Features Columns:
['date', 'tick', 'close', 'high', 'low', 'open', 'volume', 'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']


In [ ]:
# # Define a Function for adding technical indicators

# def add_features(data, feature_list, short_names):
#     """
#     Function to add technical indicators for features
#     -Takes in a dataset with Open, High, Low, Close and Volume
#     -Also takes in a list of the technical indicators to be added 
#      as well as a list of the shortened indicator names
#     """
    
#     # list of column names to filter the features
#     data_col_names = list(data.columns)
#     filter_names = data_col_names + feature_list
#     col_rename = data_col_names +  short_names
    
#     # Add technical indicators using the ta Library
#     data = add_all_ta_features(data, open="open", high="high", 
#     low="low", close="close", volume="volume") 
    
#     # Filter the Indicators with the required features
#     data = data[filter_names]
#     data.columns = col_rename # rename the columns to use shortened indicator names
#     data = data.dropna()
    
#     return data

In [ ]:
# # List of Features to add
# feature_list= ['volatility_atr','volatility_bbw','volume_obv','volume_cmf',
#                'trend_macd', 'trend_adx', 'trend_sma_fast', 
#                'trend_ema_fast', 'trend_cci', 'momentum_rsi']

# # Short names of the features
# short_names = ['atr', 'bbw','obv','cmf','macd', 'adx', 'sma', 'ema', 'cci', 'rsi']

# #feature_list= ['volatility_atr','volatility_bbw','volume_obv','volume_cmf','trend_macd']

# # Short names of the features
# #short_names = ['atr', 'bbw','obv','cmf','macd']

In [ ]:
# # Add Indicators to our dataset
# data_with_features = data.copy()
# data_with_features = add_features(data_with_features, feature_list, short_names)

In [50]:
data_with_features.head()

,date,tick,close,high,low,open,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
59,2013-01-04 00:00:00+00:00,XOM,54.466110,89.089996,88.370003,88.500000,11427900,1.430195,69.751915,-9.728804,52.079838,144.252877,2.505728,29.387219,29.940876
60,2013-01-07 00:00:00+00:00,AAPL,15.923532,18.903570,18.400000,18.642857,484156400,0.017761,69.753860,-9.740172,49.037488,-54.756636,2.444364,28.995116,29.928146
61,2013-01-07 00:00:00+00:00,AMZN,13.423000,13.486500,13.133500,13.148500,98200000,-1.288521,69.737699,-9.677461,48.845981,-66.821201,2.402473,28.705655,29.937438
62,2013-01-07 00:00:00+00:00,AVGO,2.379478,3.249000,3.210000,3.237000,16822000,-3.178243,69.738881,-9.679968,47.989728,-101.464086,2.395428,28.775179,29.937168
63,2013-01-07 00:00:00+00:00,COST,80.345329,101.730003,100.900002,101.089996,1663900,1.596923,69.547542,-9.551242,53.892606,173.424000,2.564147,30.493098,29.936111


In [67]:
feature_list = list(data_with_features.columns)[7:]

In [69]:
print(feature_list)

['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']


In [71]:
def add_cov_matrix(df):
    """
    Function to add Covariance Matrices as part of the defined states
    """
    # Sort the data and index by date and tick
    df = df.sort_values(['date', 'tick'], ignore_index=True) 
    df.index = df.date.factorize()[0]
    
    cov_list = []  # create an empty list for storing covariance matrices at each time step
    
    # lookback period for constructing the covariance matrix is one year (252 trading days)
    lookback = 252
    for i in range(lookback, len(df.index.unique())):
        data_lookback = df.loc[i - lookback:i, :]
        price_lookback = data_lookback.pivot_table(index='date', columns='tick', values='close')
        
        # Calculate percentage change without filling NaN values
        return_lookback = price_lookback.pct_change(fill_method=None).dropna()
        
        # Calculate covariance matrix of returns
        covs = return_lookback.cov().values 
        covs = covs  # you may want to normalize this with covs/covs.max() if needed
        cov_list.append(covs)
    
    # Create a DataFrame with covariance matrices and merge it back to the main DataFrame
    df_cov = pd.DataFrame({'date': df.date.unique()[lookback:], 'cov_list': cov_list})
    df = df.merge(df_cov, on='date')
    
    # Sort the DataFrame and reset index
    df = df.sort_values(['date', 'tick']).reset_index(drop=True)
    
    return df

In [73]:
# Add Covariance Matrices to our dataset
data_with_features_covs = data_with_features.copy()
data_with_features_covs = add_cov_matrix(data_with_features_covs)

In [74]:
data_with_features_covs.head()

,date,tick,close,high,low,open,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list
0,2014-01-06 00:00:00+00:00,AAPL,16.947643,19.528570,19.057142,19.194643,412610800,-1.011855,89.377971,-9.357112,48.782849,-67.903714,2.189701,38.254454,40.127613,"[[0.00031981037138226355, -3.667461313914494e-..."
1,2014-01-06 00:00:00+00:00,AMZN,19.681499,19.850000,19.421000,19.792500,63412000,-2.436377,89.382474,-9.375665,48.930880,-62.312095,2.177014,37.875218,40.123313,"[[0.00031981037138226355, -3.667461313914494e-..."
2,2014-01-06 00:00:00+00:00,AVGO,3.949247,5.340000,5.243000,5.280000,12634000,-4.779685,89.385074,-9.380973,48.103227,-101.760025,2.202519,37.994393,40.122824,"[[0.00031981037138226355, -3.667461313914494e-..."
3,2014-01-06 00:00:00+00:00,COST,93.239716,117.750000,115.889999,117.489998,2120200,0.561746,89.194390,-9.261581,52.791597,152.696814,2.327718,40.035428,40.087846,"[[0.00031981037138226355, -3.667461313914494e-..."
4,2014-01-06 00:00:00+00:00,GOOG,27.760517,27.867046,27.557707,27.721344,71037271,-0.483185,89.194313,-9.230895,49.405654,-41.813495,2.283454,38.996091,40.086442,"[[0.00031981037138226355, -3.667461313914494e-..."


In [77]:
data = data_with_features_covs

In [79]:
%store data

Stored 'data' (DataFrame)


In [81]:
data.to_csv('datasets/macd_cov_data.csv', index=False)

In [124]:
# import pandas as pd

# # Set the maximum column width to None (unlimited)
# pd.set_option('display.max_colwidth', None)

# # Now, display the first few entries of 'cov_list'
# print(train_df['cov_list'].head())


In [86]:
train_df = data
data.head()

,date,tick,close,high,low,open,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list
0,2014-01-06 00:00:00+00:00,AAPL,16.947643,19.528570,19.057142,19.194643,412610800,-1.011855,89.377971,-9.357112,48.782849,-67.903714,2.189701,38.254454,40.127613,"[[0.00031981037138226355, -3.667461313914494e-..."
1,2014-01-06 00:00:00+00:00,AMZN,19.681499,19.850000,19.421000,19.792500,63412000,-2.436377,89.382474,-9.375665,48.930880,-62.312095,2.177014,37.875218,40.123313,"[[0.00031981037138226355, -3.667461313914494e-..."
2,2014-01-06 00:00:00+00:00,AVGO,3.949247,5.340000,5.243000,5.280000,12634000,-4.779685,89.385074,-9.380973,48.103227,-101.760025,2.202519,37.994393,40.122824,"[[0.00031981037138226355, -3.667461313914494e-..."
3,2014-01-06 00:00:00+00:00,COST,93.239716,117.750000,115.889999,117.489998,2120200,0.561746,89.194390,-9.261581,52.791597,152.696814,2.327718,40.035428,40.087846,"[[0.00031981037138226355, -3.667461313914494e-..."
4,2014-01-06 00:00:00+00:00,GOOG,27.760517,27.867046,27.557707,27.721344,71037271,-0.483185,89.194313,-9.230895,49.405654,-41.813495,2.283454,38.996091,40.086442,"[[0.00031981037138226355, -3.667461313914494e-..."


In [92]:
import pandas as pd

# Assuming 'train_df' is your DataFrame
# Sample date string: "2014-01-06 00:00:00+00:00"

# Step 1: Remove the time and timezone components
train_df['date'] = train_df['date'].str.split().str[0]  # Keeps "2014-01-06"

# Step 2: Convert the 'date' column to datetime format
train_df['date'] = pd.to_datetime(train_df['date'], format='%Y-%m-%d', errors='coerce')  # Use the correct format

# Step 3: Convert 'date' to Unix timestamps
train_df['date_numeric'] = train_df['date'].astype('int64') // 10**9  # Efficient vectorized operation

# Step 4: Drop the original 'date' column if no longer needed
train_df = train_df.drop(columns=['date'])

train_df['date'] = train_df['date_numeric']

# Verify the changes
print(train_df.head())


   tick      close        high         low        open     volume      macd  \
0  AAPL  16.947643   19.528570   19.057142   19.194643  412610800 -1.011855   
1  AMZN  19.681499   19.850000   19.421000   19.792500   63412000 -2.436377   
2  AVGO   3.949247    5.340000    5.243000    5.280000   12634000 -4.779685   
3  COST  93.239716  117.750000  115.889999  117.489998    2120200  0.561746   
4  GOOG  27.760517   27.867046   27.557707   27.721344   71037271 -0.483185   

     boll_ub   boll_lb     rsi_30      cci_30     dx_30  close_30_sma  \
0  89.377971 -9.357112  48.782849  -67.903714  2.189701     38.254454   
1  89.382474 -9.375665  48.930880  -62.312095  2.177014     37.875218   
2  89.385074 -9.380973  48.103227 -101.760025  2.202519     37.994393   
3  89.194390 -9.261581  52.791597  152.696814  2.327718     40.035428   
4  89.194313 -9.230895  49.405654  -41.813495  2.283454     38.996091   

   close_60_sma                                           cov_list  \
0     40.127613 

In [94]:
# Rename 'tick' to 'tic'
train_df = train_df.rename(columns={'tick': 'tic'})

In [88]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to c:\users\user\appdata\local\temp\pip-req-build-huiuxvzt
  Resolved https://github.com/AI4Finance-LLC/FinRL-Library.git to commit ef471fcea1f3667442f5ecbf7b4c214610a5dd55
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to c:\users\user\appdata\local\temp\pip-install-crujyh9n\elegantrl_efd794ba265245e7a84377e607e00857
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 59d9a33e2b3ba2d77c052c2810bb61059736d88c
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached ccxt-3.1.60-py2.py3-none-any.whl (4.

  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-LLC/FinRL-Library.git 'C:\Users\USER\AppData\Local\Temp\pip-req-build-huiuxvzt'
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git 'C:\Users\USER\AppData\Local\Temp\pip-install-crujyh9n\elegantrl_efd794ba265245e7a84377e607e00857'
ERROR: Package 'finrl' requires a different Python: 3.11.9 not in '<3.11,>=3.10'


     ---------------------------------------  25.2/25.2 MB 8.1 MB/s eta 0:00:01
     ---------------------------------------  25.2/25.2 MB 8.1 MB/s eta 0:00:01
     ---------------------------------------  25.2/25.2 MB 8.1 MB/s eta 0:00:01
     ---------------------------------------  25.2/25.2 MB 8.1 MB/s eta 0:00:01
     ---------------------------------------  25.2/25.2 MB 8.1 MB/s eta 0:00:01
     ---------------------------------------  25.2/25.2 MB 8.1 MB/s eta 0:00:01
     ---------------------------------------  25.2/25.2 MB 8.1 MB/s eta 0:00:01
     ---------------------------------------  25.2/25.2 MB 8.1 MB/s eta 0:00:01
     ---------------------------------------  25.2/25.2 MB 8.1 MB/s eta 0:00:01
     ---------------------------------------  25.2/25.2 MB 8.1 MB/s eta 0:00:01
     ---------------------------------------  25.2/25.2 MB 8.1 MB/s eta 0:00:01
     ---------------------------------------  25.2/25.2 MB 8.1 MB/s eta 0:00:01
     -----------------------------------

In [102]:
!pip install tensorflow==1.14.0
!pip install stable-baselines[mpi]==2.10.0

ERROR: Could not find a version that satisfies the requirement tensorflow==1.14.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==1.14.0


                                              0.0/248.4 kB ? eta -:--:--
     -                                        10.2/248.4 kB ? eta -:--:--
     ---                                   20.5/248.4 kB 217.9 kB/s eta 0:00:02
     ----------                            71.7/248.4 kB 491.5 kB/s eta 0:00:01
     -------------------------------------- 248.4/248.4 kB 1.5 MB/s eta 0:00:00
                                              0.0/1.6 MB ? eta -:--:--
     --------                                 0.3/1.6 MB 10.6 MB/s eta 0:00:01
     ------------------                       0.7/1.6 MB 9.3 MB/s eta 0:00:01
     ----------------------------             1.1/1.6 MB 9.1 MB/s eta 0:00:01
     ---------------------------------------  1.6/1.6 MB 9.0 MB/s eta 0:00:01
     ---------------------------------------- 1.6/1.6 MB 7.8 MB/s eta 0:00:00


In [104]:
!pip install tensorflow

In [116]:
from finrl.test import test
print("List of Technical Indicators:")
print(INDICATORS)

ImportError: cannot import name 'INDICATORS' from 'finrl.config' (C:\Users\USER\anaconda3\Lib\site-packages\finrl\config.py)

In [106]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime

from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_portfolio import StockPortfolioEnv

from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts

import sys
sys.path.append("../FinRL-Library")

ImportError: cannot import name 'INDICATORS' from 'finrl.config' (C:\Users\USER\anaconda3\Lib\site-packages\finrl\config.py)

In [90]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,)) 
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold        
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()
            
            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))           
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions) 
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            #print(self.state)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])            
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value 
            #print("Step reward: ", self.reward)
            #self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [98]:

stock_dimension = len(train_df.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 20, State Space: 20


In [100]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)
     

NameError: name 'config' is not defined

In [ ]:
# print(data.isna().sum())  # To see if there are missing values in the dataset

In [ ]:
# import pandas as pd
# import numpy as np

# def add_cov_matrix(data):
#     """
#     Function to add Covariance Matrices as part of the defined states.
#     This function calculates the covariance matrix for a rolling window of 252 days (one year of trading days).
#     """
#     # Convert 'date' column to datetime if it's not already and remove timezone info
#     data['date'] = pd.to_datetime(data['date'], errors='coerce')  # Coerce errors to NaT (Not a Time)
    
#     # Remove timezone info if present (make it timezone-unaware)
#     data['date'] = data['date'].dt.tz_localize(None)  # Remove timezone info
    
#     # Sort the data by date and tick to ensure the order
#     data = data.sort_values(['date', 'tick'], ignore_index=True)
    
#     # Set the date column as the index for easier manipulation
#     data = data.set_index('date')
    
#     cov_list = []  # Create an empty list for storing covariance matrices
#     valid_dates = []  # List to store dates with valid covariance matrices
    
#     # Lookback period (one year = 252 trading days)
#     lookback = 252
    
#     # Loop through the dataset and calculate covariance matrices
#     for i in range(lookback, len(data.index.unique())):
#         # Select the lookback data for the last 252 days
#         data_lookback = data.loc[data.index.unique()[i - lookback:i], :]
        
#         # Create a pivot table of prices
#         price_lookback = data_lookback.pivot_table(index='date', columns='tick', values='close')
        
#         # Compute the returns (percentage change) and drop NaN values
#         return_lookback = price_lookback.pct_change(fill_method=None).dropna()
        
#         # Skip if the return data is empty or contains fewer than 2 data points
#         if return_lookback.empty or return_lookback.shape[0] < 2:
#             continue
        
#         # Compute the covariance matrix
#         covs = return_lookback.cov().values
        
#         # Skip if the covariance matrix has zero values (indicating no variation)
#         if np.any(covs == 0):
#             continue
        
#         # Append the covariance matrix and date to the respective lists
#         cov_list.append(covs)
#         valid_dates.append(data.index.unique()[i])
    
#     # Print for debugging purposes
#     print(f"Length of valid_dates: {len(valid_dates)}")
#     print(f"Length of cov_list: {len(cov_list)}")
    
#     # Ensure covariance matrices are stored as numpy arrays or appropriate format
#     df_cov = pd.DataFrame({'date': valid_dates, 'cov_list': cov_list})
    
#     # Debugging: inspect the valid_dates range
#     print("Valid dates range in df_cov:", df_cov['date'].min(), "to", df_cov['date'].max())
    
#     # Reset the index to bring 'date' back as a column for both data and df_cov
#     data = data.reset_index()
#     df_cov['date'] = pd.to_datetime(df_cov['date'])
    
#     # Debugging: inspect the first few rows of df_cov to ensure correct data
#     print(df_cov.head())
    
#     # Merge the covariance DataFrame with the original data using an outer join to handle missing dates
#     data_with_covs = data.merge(df_cov, left_on='date', right_on='date', how='outer')
    
#     # Debugging: inspect merge result before returning
#     print("Merge result head:")
#     print(data_with_covs.head())
    
#     # Reset the index and sort by date and tick
#     data_with_covs = data_with_covs.sort_values(['date', 'tick']).reset_index(drop=True)
    
#     return data_with_covs

# # Assuming 'data' is your original DataFrame
# data_with_features_covs = data.copy()  # Create a copy to preserve original data
# data_with_features_covs = add_cov_matrix(data_with_features_covs)

# # Show the result
# print(data_with_features_covs.head())  # Show first few rows of the updated data


In [ ]:
# data_with_features_covs.head()

In [ ]:
# data = data_with_features_covs

In [ ]:
# %store data